In [17]:
from nba_api.stats.endpoints import leaguegamelog
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from termcolor import colored
import requests
import mysql.connector
import csv_getters.helper_functions as hf
from csv_getters.config import api_basketball_key, conn_host, conn_database, conn_user, conn_password
import datetime as dt
from IPython.display import clear_output
from dateutil import tz
import json
import time
from nba_api.stats.static import teams 

In [2]:
season = '2022-2023'
season_split = int(season.split('-')[0])
now =  time.time()
date = dt.datetime.now().strftime('%Y-%m-%d')

min_games = 5
totals_n_last_games = 15
n_last_games = 10
n_last_specific_games = 5

from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/Sao_Paulo')

pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
def connect_to_db():
    return mysql.connector.connect(host=conn_host, 
                                     database=conn_database,
                                     user=conn_user,
                                     password=conn_password)

def execute_query(query, read_only = True):
    resp = None
    try:
        db = connect_to_db()
        if read_only:
            resp = pd.read_sql_query(query, db)
        else:
            mycursor = db.cursor()
            mycursor.execute(query)

            db.commit()
    except Exception as e:
        print(e)
    db.close()
    return resp

def execute_multiple_queries(queries):
    try:
        db = connect_to_db()
        mycursor = db.cursor()
        for query in queries:
            mycursor.execute(query)

        db.commit()
        db.close()
    except Exception as e:
        print(e)

In [4]:
def add_match_info_to_db(home_game, away_game, winner):
    query = (f"INSERT IGNORE INTO games (id, date, season, is_playoff, winner, " + 
                  "home_id, home_pts, home_fgm, home_fga, home_fg_pct, home_fg3m, home_fg3a, home_fg3_pct, home_ftm, home_fta, home_ft_pct, home_oreb, home_dreb, home_reb, home_ast, home_stl, home_blk, home_tov, home_pf," +
                  "away_id, away_pts, away_fgm, away_fga, away_fg_pct, away_fg3m, away_fg3a, away_fg3_pct, away_ftm, away_fta, away_ft_pct, away_oreb, away_dreb, away_reb, away_ast, away_stl, away_blk, away_tov, away_pf" +
                  f") VALUES ({home_game['GAME_ID']}, '{home_game['GAME_DATE']}', {get_season_year(home_game['SEASON_ID'])}, {home_game['IS_PLAYOFFS']}, '{winner}', " + 
                  f"{home_game['TEAM_ID']}, {home_game['PTS']}, {home_game['FGM']}, {home_game['FGA']}, {home_game['FG_PCT']}, {home_game['FG3M']}, {home_game['FG3A']}, {home_game['FG3_PCT']}, {home_game['FTM']}, {home_game['FTA']}, {home_game['FT_PCT']}, {home_game['OREB']}, {home_game['DREB']}, {home_game['REB']}, {home_game['AST']}, {home_game['STL']}, {home_game['BLK']}, {home_game['TOV']}, {home_game['PF']}, " +
                  f"{away_game['TEAM_ID']}, {away_game['PTS']}, {away_game['FGM']}, {away_game['FGA']}, {away_game['FG_PCT']}, {away_game['FG3M']}, {away_game['FG3A']}, {away_game['FG3_PCT']}, {away_game['FTM']}, {away_game['FTA']}, {away_game['FT_PCT']}, {away_game['OREB']}, {away_game['DREB']}, {away_game['REB']}, {away_game['AST']}, {away_game['STL']}, {away_game['BLK']}, {away_game['TOV']}, {away_game['PF']}" +
                  ")")
    return query

def add_player_to_db(player_id, name):
    query = (f"INSERT IGNORE INTO players (id, name) VALUES ({player_id}, '{name}')")
    return query
    
def add_player_game_to_db(game):
    query = (f"INSERT IGNORE INTO playergames (team_id, player_id, game_id, minutes, pts, fgm, fga, fg_pct, fg3m, fg3a, fg3_pct, ftm, fta, ft_pct, oreb, dreb, reb, ast, stl, blk, tov, pf, plus_minus) " + 
                  f"VALUES ({game['TEAM_ID']}, {game['PLAYER_ID']}, {game['GAME_ID']}, {game['MIN']}, {game['PTS']}, {game['FGM']}, {game['FGA']}, {game['FG_PCT']}, {game['FG3M']}, {game['FG3A']}, {game['FG3_PCT']}, {game['FTM']}, {game['FTA']}, {game['FT_PCT']}, {game['OREB']}, {game['DREB']}, {game['REB']}, {game['AST']}, {game['STL']}, {game['BLK']}, {game['TOV']}, {game['PF']}, {game['PLUS_MINUS']})")
    return query

def get_season(season, season_type='Regular Season'):
    season_games = leaguegamelog.LeagueGameLog(season = str(season), season_type_all_star=season_type).get_data_frames()[0]
    season_players = leaguegamelog.LeagueGameLog(season = str(season), player_or_team_abbreviation = 'P', season_type_all_star=season_type).get_data_frames()[0]
    season_games['IS_PLAYOFFS'] = True if season_type == 'Playoffs' else False
    season_players['IS_PLAYOFFS'] = True if season_type == 'Playoffs' else False
    
    season_games.dropna(subset=['FG_PCT','FT_PCT','FG3_PCT'], inplace=True)

    season_games['GAME_ID'] = pd.to_numeric(season_games['GAME_ID'])
    season_players['GAME_ID'] = pd.to_numeric(season_players['GAME_ID'])

    season_games['GAME_DATE'] = pd.to_datetime(season_games['GAME_DATE'])
    season_players['GAME_DATE'] = pd.to_datetime(season_players['GAME_DATE'])

    season_games = season_games.sort_values(['GAME_DATE', 'GAME_ID'], ascending=[True, True]).reset_index(drop=True)
    return season_games, season_players

def get_season_year(season_id):
    return int(str(season_id)[1:])

In [5]:
season_games, season_players = get_season(season_split, season_type='Regular Season')

In [6]:
season_games.tail()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,IS_PLAYOFFS
229,22022,1610612742,DAL,Dallas Mavericks,22200115,2022-11-02,DAL vs. UTA,W,240,42,...,37,20,9,4,12,14,103,3,1,False
230,22022,1610612763,MEM,Memphis Grizzlies,22200116,2022-11-02,MEM @ POR,W,240,38,...,46,28,7,5,15,19,111,5,1,False
231,22022,1610612757,POR,Portland Trail Blazers,22200116,2022-11-02,POR vs. MEM,L,240,36,...,46,27,5,3,20,17,106,-5,1,False
232,22022,1610612747,LAL,Los Angeles Lakers,22200117,2022-11-02,LAL vs. NOP,W,265,45,...,56,30,6,8,18,17,120,3,1,False
233,22022,1610612740,NOP,New Orleans Pelicans,22200117,2022-11-02,NOP @ LAL,L,265,47,...,51,35,10,5,14,20,117,-3,1,False


In [7]:
season_players.tail()

,SEASON_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,...,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FANTASY_PTS,VIDEO_AVAILABLE,IS_PLAYOFFS
2573,22022,1631207,Dalen Terry,1610612741,CHI,Chicago Bulls,22200111,2022-11-02,CHI vs. CHA,W,...,0,0,0,0,0,0,0,1.2,1,False
2574,22022,1630547,James Bouknight,1610612766,CHA,Charlotte Hornets,22200111,2022-11-02,CHA @ CHI,L,...,0,0,1,0,1,7,-8,13.6,1,False
2575,22022,1630539,Kai Jones,1610612766,CHA,Charlotte Hornets,22200111,2022-11-02,CHA @ CHI,L,...,0,0,1,0,0,2,2,5.0,1,False
2576,22022,1630550,JT Thor,1610612766,CHA,Charlotte Hornets,22200111,2022-11-02,CHA @ CHI,L,...,1,1,1,0,1,4,-10,13.9,1,False
2577,22022,1628372,Dennis Smith Jr.,1610612766,CHA,Charlotte Hornets,22200111,2022-11-02,CHA @ CHI,L,...,6,2,0,1,0,10,0,30.0,1,False


In [8]:
games_to_insert_queries = []
players_to_insert_queries = []
player_games_to_insert_queries = []

season_id = ''
for i, g in season_games.groupby(season_games.index // 2):
    clear_output(wait=True)
    print("{}/{}".format(i, len(season_games.index) // 2))
    if g.iloc[[0],:].iloc[0]['WL'] == None:
        break
        
    if '@' in g.iloc[[0],:].iloc[0]['MATCHUP']:
        away_game = g.iloc[0,:]
        home_game = g.iloc[1,:]
        winner = 'H' if g.iloc[1,:]['WL'] == 'W' else 'A'
    else:
        home_game = g.iloc[0,:]
        away_game = g.iloc[1,:]
        winner = 'H' if g.iloc[0,:]['WL'] == 'W' else 'A'
    
    game_players = season_players.loc[season_players['GAME_ID'] == home_game['GAME_ID']]
    game_players = game_players.replace({np.nan: 0})
    
    games_to_insert_queries.append(add_match_info_to_db(home_game, away_game, winner))
    
    for index, player in game_players.iterrows():
        players_to_insert_queries.append(add_player_to_db(player['PLAYER_ID'], player['PLAYER_NAME'].replace("'", "")))
        player_games_to_insert_queries.append(add_player_game_to_db(player))

116/117


In [9]:
execute_multiple_queries(games_to_insert_queries)
execute_multiple_queries(players_to_insert_queries)
execute_multiple_queries(player_games_to_insert_queries)

In [19]:
teams_list = teams.get_teams()
teams_df = pd.DataFrame(teams_list)

In [20]:
teams_df.head()

,id,full_name,abbreviation,nickname,city,state,year_founded
0,1610612737,Atlanta Hawks,ATL,Hawks,Atlanta,Atlanta,1949
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
2,1610612739,Cleveland Cavaliers,CLE,Cavaliers,Cleveland,Ohio,1970
3,1610612740,New Orleans Pelicans,NOP,Pelicans,New Orleans,Louisiana,2002
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966


In [10]:
season_games = execute_query(f"SELECT g.id, g.date, g.season, g.is_playoff, g.winner, g.home_id, ht.name as home_name, g.home_pts, g.home_fgm, g.home_fga, g.home_fg_pct, g.home_fg3m, g.home_fg3a, g.home_fg3_pct, g.home_ftm, g.home_fta, g.home_ft_pct, g.home_oreb, g.home_dreb, g.home_reb, g.home_ast, g.home_stl, g.home_blk, g.home_tov, g.home_pf, g.away_id, at.name as away_name, g.away_pts, g.away_fgm, g.away_fga, g.away_fg_pct, g.away_fg3m, g.away_fg3a, g.away_fg3_pct, g.away_ftm, g.away_fta, g.away_ft_pct, g.away_oreb, g.away_dreb, g.away_reb, g.away_ast, g.away_stl, g.away_blk, g.away_tov, g.away_pf, g.home_odds, g.away_odds, g.over_under_line, g.spread_line FROM games AS g LEFT JOIN teams as ht ON g.home_id = ht.id LEFT JOIN teams as at ON g.away_id = at.id WHERE g.season >= {season_split - 5} and g.season <= {season_split} ORDER BY g.date ASC")
season_games_plyrs = execute_query(f"SELECT g.id as game_id, g.date, g.season, g.is_playoff, g.winner, g.home_id, g.away_id, pg.team_id, pg.player_id, pg.minutes, pg.pts, pg.fgm, pg.fga, pg.fg_pct, pg.fg3m, pg.fg3a, pg.fg3_pct, pg.ftm, pg.fta, pg.ft_pct, pg.oreb, pg.dreb, pg.reb, pg.ast, pg.stl, pg.blk, pg.tov, pg.pf, pg.plus_minus FROM playergames AS pg LEFT JOIN games as g on pg.game_id = g.id WHERE g.season >= {season_split - 5} and g.season = {season_split} ORDER BY g.date ASC")

C:\Users\caiog\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
season_games_plyrs.head()

In [ ]:
season_games['home_off_rtg'] = season_games.apply(lambda row: hf.get_team_offensive_rating_game(row, 'H'), axis = 1)
season_games['home_def_rtg'] = season_games.apply(lambda row: hf.get_team_offensive_rating_game(row, 'H'), axis = 1)

season_games['away_off_rtg'] = season_games.apply(lambda row: hf.get_team_offensive_rating_game(row, 'A'), axis = 1)
season_games['away_def_rtg'] = season_games.apply(lambda row: hf.get_team_offensive_rating_game(row, 'A'), axis = 1)

In [ ]:
season_games.head()

In [13]:
def get_season_fixtures(season):
    headers = {
        'X-RapidAPI-Key': api_basketball_key,
        'X-RapidAPI-Host': 'api-basketball.p.rapidapi.com'
    }
    response = requests.get(f"https://api-basketball.p.rapidapi.com/games?league=12&season={season}&timezone=America/Sao_Paulo&date={date}", headers=headers)
    response_parsed = json.loads(response.text)
    return response_parsed['response']

In [14]:
fixtures = get_season_fixtures(season)
fixtures = [fixture for fixture in fixtures if fixture['timestamp'] >= now and fixture['timestamp'] <= (now + 24*60*60)]
fixtures = sorted(fixtures, key = lambda x: x['timestamp'])

In [15]:
def get_team_made_conceded_pct(games, line):
    made = np.array(games['team_pts'])
    conceded = np.array(games['opp_pts'])
    totals = made + conceded
    greater = (totals > line).sum()
    pct = greater*100/len(totals)
    return pct

In [ ]:
def get_team_elo(team):
    team = team.lower().replace(" ", "")
    if team == '76ers':
        team = 'sixers'
    contents = urllib.request.urlopen("https://projects.fivethirtyeight.com/complete-history-of-the-nba/data/{}.json".format(team)).read()
    contents = json.loads(contents)
    return contents['value'][-1]['y']

In [ ]:
def get_team_previous_games(game_date, team, opp_id, line, scenario):  
    team_id = team['id']
    
    response = hf.get_team_previous_games(season_games, team_id, game_date, season_split)
    if not response: return None
    
    home_previous_games, away_previous_games, previous_games, previous_season_games, home_previous_season_games, away_previous_season_games = response
    
    if len(previous_season_games.index) < 10:
        return None
    
    last_n_games = previous_season_games.iloc[-n_last_games:,:]
    
    # Totals info
    totals_overall_pct = get_team_made_conceded_pct(previous_season_games.iloc[-totals_n_last_games:,:], line)
    if scenario == 'H':
        totals_ha_pct = get_team_made_conceded_pct(home_previous_season_games.iloc[-totals_n_last_games:,:], line)
    else:
        totals_ha_pct = get_team_made_conceded_pct(away_previous_season_games.iloc[-totals_n_last_games:,:], line)
    
    # Get last game ELO
    last_game_date = str(previous_season_games.iloc[-1,:]['date'])
    elo = get_team_elo(team['nickname'])
    
    # Last n games pct
    pct_last_n_games = hf.get_wl_pct(last_n_games)[0]
    
    # Getting Previous A x B Matchups
    last_matchups = previous_games[previous_games['opp_id'] == opp_id].iloc[-10:,:]
    
    # Getting player information
    team_per = hf.get_team_per_mean(team_id, game_id, game_date, season, season_games_plyrs)
    
    # Season Win Percentage
    season_pct = hf.get_wl_pct(previous_season_games)[0]
    
    # Last n/2 games pct and Season H/A Win Percentage
    if scenario == 'H':
        ha_pct_last_n_games = hf.get_wl_pct(home_previous_season_games.iloc[-n_last_specific_games:,:])[0]
        ha_pct = hf.get_wl_pct(home_previous_season_games)[0]
    else:
        ha_pct_last_n_games = hf.get_wl_pct(away_previous_season_games.iloc[-n_last_specific_games:,:])[0]
        ha_pct = hf.get_wl_pct(away_previous_season_games)[0]
    
    # Matchup Win Percentage
    matchup_pct = hf.get_wl_pct(last_matchups)[0]
    
    # Calculating Current Streak
    streak = hf.current_streak(previous_season_games)
    
    stats_team = hf.get_team_stats (last_n_games, season_pct, teams_per[team_id], elo, matchup_pct, ha_pct, streak, pct_last_n_games, ha_pct_last_n_games, totals_overall_pct, totals_ha_pct)
    
    return stats_team

In [16]:
for fixture in fixtures:
    game_date = np.datetime64(dt.datetime.utcfromtimestamp(fixture['timestamp']).replace(tzinfo=from_zone).astimezone(to_zone).strftime('%Y-%m-%d %H:%M:%S'))
    home_name, away_name = fixture['teams']['home']['name'], fixture['teams']['away']['name']
    
    home = teams_df.loc[teams['full_name'] == home_name].iloc[0,0]
    away = teams_df.loc[teams['full_name'] == away_name].iloc[0,0]
    
    print('\n')
    
    while True:
        line = input(f"Type the line that represents the totals line of the game {away_name} @ {home_name}: ").replace(',','.')
        try:
            line = float(line)
            break
        except ValueError:
            pass
        print("Invalid value.")
        
    stats_a = get_team_previous_games(game_date, home, away['id'], line, 'H')
    
    if not stats_a: continue

    home_overall_pct, home_ha_pct = stats_a[-2], stats_a[-1]

    stats_b = get_team_previous_games(game_date, away, home['id'], line, 'A')
    if not stats_b: continue

    away_overall_pct, away_ha_pct = stats_b[-2], stats_b[-1]
    
    print("\n{} had their totals above {} in {:.2f}% of the last {} games".format(home_name, line, home_overall_pct, n_last_games))
    print("At home, {} had their totals above {} in {:.2f}% of the last {} games".format(home_name, line, home_ha_pct, n_last_games))
    
    print("\n{} had their totals above {} in {:.2f}% of the last {} games".format(away_name, line, away_overall_pct, n_last_games))
    print("On the road, {} had their totals above {} in {:.2f}% of the last {} games".format(away_name, line, away_ha_pct, n_last_games))
    
    over = min(home_overall_pct, home_ha_pct, away_overall_pct, away_ha_pct) > 50
    under = max(home_overall_pct, home_ha_pct, away_overall_pct, away_ha_pct) < 50
    if over or under:
        print(colored(f"GOOD BET: {'OVER' if over else 'UNDER'}", 'green'))
    else:
        print(colored("BAD BET", 'red'))



Type the line that represents the totals line of the game Golden State Warriors @ Orlando Magic: 226,5

Orlando Magic had their totals above 226.5 in 12.50% of the last 15 games
At home, Orlando Magic had their totals above 226.5 in 50.00% of the last 15 games

Golden State Warriors had their totals above 226.5 in 87.50% of the last 15 games
On the road, Golden State Warriors had their totals above 226.5 in 75.00% of the last 15 games
BAD BET


Type the line that represents the totals line of the game Denver Nuggets @ Oklahoma City Thunder: 228,5

Oklahoma City Thunder had their totals above 228.5 in 14.29% of the last 15 games
At home, Oklahoma City Thunder had their totals above 228.5 in 0.00% of the last 15 games

Denver Nuggets had their totals above 228.5 in 57.14% of the last 15 games
On the road, Denver Nuggets had their totals above 228.5 in 75.00% of the last 15 games
BAD BET
